In [1]:
from cuml.svm import SVC, LinearSVC
import cudf
#from cuml.model_selection import train_test_split
import cupy as cp
from cuml.metrics import accuracy_score
from cuml.pipeline import make_pipeline
from cuml.decomposition import PCA

from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
#from sklearn.svm import SVC
#from sklearn.metrics import accuracy_score
import numpy as np
from itertools import product
#from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from matplotlib.lines import Line2D
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
data = fetch_openml('mnist_784', version=1)

Xdata, ydata = data.data, data.target
Xdata = Xdata.astype(np.float32)
ydata = ydata.astype(np.float32)

Xtrain, Xtemp, ytrain, ytemp = train_test_split(Xdata, ydata, test_size=0.3, random_state=42, stratify=ydata)
Xval, Xtest, yval, ytest = train_test_split(Xtemp, ytemp, test_size=0.4, random_state=42, stratify=ytemp)

Xtrain_cp = cp.asarray(Xtrain)
ytrain_cp = cp.asarray(ytrain)
Xval_cp = cp.asarray(Xval)
yval_cp = cp.asarray(yval)

In [3]:
# hyperparameters

grid_params = {
    'C': [0.1, 1, 10],
    #'gamma': ['scale', 'auto', 0.01],
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

# Model Training

In [4]:
best_model = None
best_params = None
best_val_acc = 0
val_accs = []

In [5]:
param_combinations = product(grid_params['C'], grid_params['gamma'], grid_params['kernel'])

for C, gamma, kernel in param_combinations:
    if kernel == 'linear':
        svm = LinearSVC(C=C, probability=False, max_iter=10000)
    else:
        svm = SVC(C=C, gamma=gamma, kernel=kernel, probability=False)

    svm.fit(Xtrain_cp, ytrain_cp)
    pred = svm.predict(Xval_cp)
    val_acc = accuracy_score(yval_cp, pred)
    val_accs.append(val_acc)
    # print(f"Params: C={C}, gamma={gamma}, kernel={kernel} | Evaluation Accuracy: {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = svm
        best_params = {'C': C, 'gamma': gamma, 'kernel': kernel}

print("Best hyperparameters:", best_params)
print("HIghest validation accuracy:", best_val_acc)

[2025-02-24 14:23:08.482] [CUML] [warning] L-BFGS: max iterations reached
[2025-02-24 14:23:08.482] [CUML] [warning] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
Best hyperparameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
HIghest validation accuracy: 0.9812698364257812


In [22]:
# test_pred = best_model.predict(Xtest)
# test_acc = accuracy_score(ytest, test_pred)
# print(f"Final Test Accuracy: {test_acc:.4f}")
